<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/Resnet50_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 lib 불러오기

In [2]:
# 모델 로드 
# top 그대로 유지해서 할꺼면, 같은 이미지에 대해서 테스트를 진행 해야함

In [3]:
# 데이터 로드 , 데이터 기본 제공 말고, 추가적인 거는 내가 모양 맞춰서 따로 정제 해야함

In [4]:
#https://github.com/Bao-Jiarong/VGG/blob/master/vgg.py
# 여기꺼 참고하면 좋을거 같음

In [12]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [6]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [7]:
print(raw_train_x.shape)
print(raw_train_y.shape)
print(raw_test_x.shape)
print(raw_test_y.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [8]:
# float을 넣을거면 0~1 사이 값으로 바꿔야함
train_x = raw_train_x/255
test_x = raw_test_x/255
train_y = raw_train_y
test_y = raw_test_y
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

"""
def show_sample(i):
  print(raw_train_y[i][0], labels[raw_train_y[i][0]])
  plt.imshow(raw_train_x[i])
  plt.show()

for i in [2, 10, 12, 14]:
  show_sample(i)
"""

'\ndef show_sample(i):\n  print(raw_train_y[i][0], labels[raw_train_y[i][0]])\n  plt.imshow(raw_train_x[i])\n  plt.show()\n\nfor i in [2, 10, 12, 14]:\n  show_sample(i)\n'

In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models
from tensorflow.keras import layers
from tf.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

Resnet = ResNet50(weights = 'imagenet', #  그냥 초기화 하는거면, 이거 지우기
            include_top = False,
            input_shape=(32,32,3))
Resnet.summary()

# vgg conv 구조만 사용하고 마지막 FC layer는 다른거 사용
model = models.Sequential()
model.add(Resnet)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(10, activation = "sigmoid"))
model.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                          

In [11]:
# optimizer, loss 함수를 정의하고,  학습 준비를 한다,  metrics 는 어떤 일이 발생하는지 보여줄 것들
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 한번에 몇개의 데이터 학습하고 가중치 갱신할지 
model.fit(train_x, train_y, epochs=1, verbose=1, batch_size=128)

313/313 [==============================] - 38s 119ms/step - loss: 4.1227 - accuracy: 0.1204
loss= 4.122739791870117
acc= 0.12039999663829803


NameError: ignored

In [13]:
loss, acc = model.evaluate(test_x, test_y)
print("loss=",loss)
print("acc=",acc)

y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=1)

print(predicted)

313/313 [==============================] - 37s 119ms/step - loss: 4.1227 - accuracy: 0.1204
loss= 4.122739791870117
acc= 0.12039999663829803
[2 0 0 ... 0 0 2]


In [15]:
model.save('Resnet_h5_model.h5')
model.save('Resnet_another_type')

INFO:tensorflow:Assets written to: Resnet_another_type/assets


In [16]:
!pip install tensorflow_model_optimization

     |████████████████████████████████| 237 kB 23.9 MB/s 


In [17]:
from keras.models import load_model
import tensorflow_model_optimization as tfmot

test = load_model('Resnet_h5_model.h5')
# 전체 양자화 해주기
quant_aware_model_all = tfmot.quantization.keras.quantize_model(test)
quant_aware_model_all.summary()

ValueError: ignored